In [1]:
# Python imports
import requests
import zipfile
import pandas as pd 
import os
import glob
from memory_profiler import memory_usage 

# Jupyter Lab cell extensions
%load_ext rpy2.ipython 
%load_ext memory_profiler 

### Downloading the data

In [2]:
# Create data directories if they don't exist
output_raw_dir = "../data/raw"
output_proc_dir = "../data/processed"
os.makedirs(output_raw_dir, exist_ok=True)
os.makedirs(output_proc_dir, exist_ok=True)

# Retrieve the file id for the rainfall data and download as a zip
file_id = 26766812
response = requests.get(f'https://api.figshare.com/v2/file/download/{file_id}')
fh = open(output_raw_dir + '/data.zip', 'wb')
fh.write(response.content)
fh.close()

# Extract the zip file
with zipfile.ZipFile(output_raw_dir + '/data.zip', 'r') as zip_ref:
    zip_ref.extractall(output_raw_dir + '/data_extracted')

### Combining data CSVs

In [3]:
%%time
%memit
output_combined_filepath = output_proc_dir + "/combined_data.csv"

# Gather extracted .csv files and add a model identification column
csvfiles = [file for file in os.listdir(output_raw_dir + "/data_extracted") if ".csv" in file]
frame_list = []
for csvfile in csvfiles:
    df = pd.read_csv(output_raw_dir + "/data_extracted/" + csvfile, index_col=None, header=0)
    df['model'] = csvfile.split("_")[0] 
    frame_list.append(df)

# Combine as a single dataframe
combined_df = pd.concat(frame_list, axis=0, ignore_index=True)

# Save combined data frame to disk
combined_df.to_csv(output_combined_filepath)
print(output_combined_filepath + " file saved!")
combined_df.head()

peak memory: 1007.22 MiB, increment: 0.21 MiB
../data/processed/combined_data.csv file saved!
CPU times: user 7min 55s, sys: 22.9 s, total: 8min 18s
Wall time: 8min 34s


,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


#### Run time and memory Usage Discussion:

peak memory: 1061.02 MiB, increment: 0.00 MiB
Wall time: 1min 51s

peak memory: 924.72 MiB, increment: 0.08 MiB
CPU times: user 48 s, sys: 5.18 s, total: 53.2 s
Wall time: 55.6 s

peak memory: 935.61 MiB, increment: 0.11 MiB
CPU times: user 56.5 s, sys: 3.65 s, total: 1min
Wall time: 1min

peak memory: 1018.20 MiB, increment: 0.19 MiB
CPU times: user 1min 2s, sys: 12 s, total: 1min 14s
Wall time: 1min 20s

#### Discussion:

Running the combining data process is very RAM intensive. It's important to verify we have enough RAM and hard disk space available before runnning the notebook. Otherwise, the jupyter notebook will crash. We ran the cell above, and our computers took around 1min to run and the peak memory usage was around 1000 MB.

### Load the combined CSV to memory and perform a simple EDA
1. Investigate at least two of the following approaches to reduce memory usage while performing the EDA (e.g., value_counts).
* Changing dtype of your data
* Load just columns what we want
* Loading in chunks
* Dask

2. Discuss your observations.

#### Loading in chunks for counting the model column

In [4]:
%%time
%%memit
import dask.dataframe as dd

model_counts = pd.Series(dtype=int)
for chunk in pd.read_csv(output_combined_filepath, chunksize=10_000_000):
    model_counts = model_counts.add(chunk["model"].value_counts(), fill_value=0)
print(model_counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
observed              46020
dtype: int64
peak memory: 8162.10 MiB, increment: 0.06 MiB
CPU times: user 1min 16s, sys: 10.2 s, total: 1min 26s
Wall time: 1min 28s


#### Loading only the column we want for the EDA, which is the model column

In [5]:
%%time
%%memit
data_model = pd.read_csv(output_combined_filepath, usecols=["model"])
print(data_model["model"].value_counts())

MPI-ESM1-2-HR       5154240
NorESM2-MM          3541230
TaiESM1             3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
CMCC-CM2-HR4        3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
observed              46020
Name: model, dtype: int64
peak memory: 2870.74 MiB, increment: 589.55 MiB
CPU times: user 35.3 s, sys: 3.31 s, total: 38.6 s
Wall time: 39.7 s


#### Loading using Dask

In [6]:
%%time
%%memit

ddf = dd.read_csv(output_combined_filepath)
print(ddf["model"].value_counts().compute())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
NorESM2-MM          3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
NorESM2-LM           919800
CanESM5              551880
BCC-ESM1             551880
observed              46020
Name: model, dtype: int64
peak memory: 3520.40 MiB, increment: 1015.82 MiB
CPU times: user 1min 40s, sys: 22 s, total: 2min 2s
Wall time: 52.2 s


### Discussion

In this section, we experimented with loading using chunk size of 10,000,000, loading using only the column we wanted, and loading using Dask.  

Comparing the results:
* Loading using chunk size of 10,000,000:  
peak memory: 3078.15 MiB, increment: 2934.44 MiB
Wall time: 1min 59s

* Loading using only the column we wanted:  
peak memory: 1424.96 MiB, increment: 953.43 MiB
Wall time: 1min 1s

* Loading using Dask:  
peak memory: 1943.04 MiB, increment: 995.03 MiB
Wall time: 1min 2s

From the above runtime stats, we can observe that loading using chunk size takes the most of the memory and longer wall time comparing to loading using only the column we wanted and loading using Dask. Loading using only the column we wanted has the minimal number of peak memory usage and wall time comparing to loading using Dask.


### Perform a simple EDA in R

Pick an approach to transfer the dataframe from python to R.
* Parquet file
* Feather file (We chose this one)
* Pandas exchange
* Arrow exchange

### Discuss why you chose this approach over others.

From the lecture notes, feather is faster than the parquet or arrow format in writing to files and it's convenient to use. It's very common to use feather with the R programming language because the API for reading and writing is simple to understand and close to the base R file reading and writing syntax. We also referred to [this article](https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d) before choosing this format, which performed a variety of benchmark comparison. From the experimentations of different file formats, the comparisons show that feather format is an ideal choice to store the data between Jupyter notebook sessions. Using feather format will result in high I/O speed and it doesn’t take huge memory on the computer disk; it also doesn’t need any unpacking when loaded the data back into RAM.

In [7]:
# Load arrow, parquet, and feather packages
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.parquet as pq
import rpy2.rinterface
import rpy2_arrow.pyarrow_rarrow as pyra
import pyarrow.feather as feather

In [8]:
%%R
#just seeing if its available
library("arrow")
library("dplyr")

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [9]:
%%time
%%memit

dataset = ds.dataset(output_combined_filepath, format="csv")
table = dataset.to_table()

peak memory: 3227.55 MiB, increment: 1248.86 MiB
CPU times: user 26.6 s, sys: 14.6 s, total: 41.3 s
Wall time: 36.4 s


In [10]:
%%time

# experiment in writing in feather format 
output_feather_filepath = output_proc_dir + '/combined_data.feather'
feather.write_feather(table, output_feather_filepath)

CPU times: user 7.72 s, sys: 17.1 s, total: 24.8 s
Wall time: 13.4 s


In [13]:
%%time
%%R

# Load R imports
library(arrow)
library(dplyr)

### here we are showing how much time it took to read a feather file what we wrote in python
start_time <- Sys.time()

output_feather_filepath = '../data/processed/combined_data.feather'
r_table <- arrow::read_feather(output_feather_filepath, col_select=c("model"))
print(class(r_table))

result <- count(r_table, model)

end_time <- Sys.time()
print(end_time - start_time)
print(result)

[1] "tbl_df"     "tbl"        "data.frame"
Time difference of 8.194424 secs
# A tibble: 28 x 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# … with 18 more rows
CPU times: user 5.88 s, sys: 2.77 s, total: 8.65 s
Wall time: 8.65 s
